In [102]:
import json
import os
from datetime import datetime

import yfinance as yf
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults
from IPython.display import Markdown

In [103]:
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return stock

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetches stock prices for {ticket} from the last year about a specific company from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [104]:
os.environ["OPENAI_API_KEY"] = "key"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [105]:
stockPriceAnalyst = Agent(
  role= "Senior stock price Analyst",
  goal= "Find the {ticket} stock price and analyses trends",
  backstory= """You are a highly experienced in analyzing the price of an specific stock and make predictions about its future price.""",
  verbose= True,
  llm= llm,
  max_iter= 5,
  memory= True,
  tools= [yahoo_finance_tool],
  allow_delegation=False
)

In [106]:
getStockPrice = Task(
  description= "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
  expected_output= """ Specify the current trend stock price - up, down or sideways,
  eg. stock= 'APP, price UP'
  """,
  agent= stockPriceAnalyst
)

In [107]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [108]:
newsAnalyst = Agent(
  role= "Stock News Analyst",
  goal= """Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with 
  the news context. For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.
  """,
  backstory= """ You are a highly experienced in analyzing the market trends and news and have tracked assets for more then 10 years.
  You are also master level analyst in the tradicional markets and have deep understanding of human psychology.
  You understand news, theirs tittles and information, but you look at those with a health dose of skepticism.
  You consider also the source of the news articles.
  """,
  verbose= True,
  llm= llm,
  max_iter= 10,
  memory= True,
  tools= [search_tool],
  allow_delegation=False
)

In [109]:
get_news = Task(
  description= """Take the stock and always inlude BTC to it (if not request).
  Use the search tool to search each one individually.
  The current date is {datetime.now()}.
  Compose the results into a helpfull report
  """,
  expected_output= """ A summary of the overall market and one sentece summary for each request asset.
  Include a fear/greed score for each asset based on the news. Use format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent= newsAnalyst
)

In [110]:
stockAnalystWrite = Agent(
  role= "Senior Stock Analyst Writter",
  goal= """ Analyze the trends price and news and write an insighfull compelling and informative 3 paaragraph long newsletter based on the stock report and price trend.""",
  backstory= """ You are widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
  and narratives that resonate with wider audiences.
  You anderstand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses.
  You are able to hold multiple opinions when analysing anything.
  """,
  verbose= True,
  llm= llm,
  max_iter= 5,
  memory= True,
  allow_delegation=True
)

In [111]:
writeAnalyses = Task(
  description= """ Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
  that is brief and highlights the most important points.
  Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
  Include the  previous analyses of stock trend and news summary.
  """,
  expected_output= """ An eloquent 3 paragraphs newsletter formated as markdown in an east readable manner. it should contain:
  - 3 bullets executive summary
  - Introduction - set the overall picture and spike up the interest
  - main part provides the meat of the analysis including the news summary and fear/greed scores
  -summary - key facts and concrete future trend prediction - up, down or sideways.
  """,
  agent= stockAnalystWrite,
  context= [getStockPrice, get_news]
)

In [112]:
crew = Crew(
  agents= [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
  tasks= [getStockPrice, get_news, writeAnalyses],
  process= Process.hierarchical,
  full_output= True,
  share_crew=False,
  manager_llm=llm,
  max_iter=15
)

2024-08-22 18:43:26,683 - 30252 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [113]:
results = crew.kickoff()

In [114]:
Markdown(results['final_output'])

Agent stopped due to iteration limit or time limit.
